<a href="https://colab.research.google.com/github/zghnjkt/tabletennis/blob/main/csv_to_word_TTclub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_LINE_SPACING
import os
import re

'''
아마 지금 서식으로 할 듯 24/07/01
돌리기 전에 저장할 위치 가져올 파일위치 설정, 저장할 위치 설정
csv파일은 '니퐁내퐁 8기 신청서(응답) - 설문지 응답 시트1' 형식으로 할 듯
'''
def sanitize_filename(filename):
    # 파일 이름에 사용할 수 없는 문자들을 제거합니다
    return re.sub(r'[\\/*?:"<>|]', "", filename)

def sanitize_cell_content(content):
    # 셀 내용에서 괄호와 따옴표를 제거합니다
    sanitized_content = re.sub(r'[\"\'\(\)]', "", str(content))
    return sanitized_content

def set_paragraph_format(paragraph, line_spacing=1, font_size=10):
    # 문단 형식을 설정합니다
    paragraph_format = paragraph.paragraph_format
    paragraph_format.line_spacing = line_spacing  # 줄 간격 설정

    # 글꼴 크기 설정
    for run in paragraph.runs:
        run.font.size = Pt(font_size)

def csv_rows_to_words(csv_file, output_dir):
    # CSV 파일을 읽습니다
    df = pd.read_csv(csv_file)

    # 첫 번째 열을 삭제합니다
    df = df.drop(df.columns[0], axis=1)

    # 출력 디렉터리가 존재하지 않으면 생성합니다
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 각 행을 하나의 Word 파일로 저장합니다
    for index, row in df.iterrows():
        try:
            title_parts = [str(row[0]), str(row[4]), str(row[2])]
            title = ' '.join(title_parts)
        except IndexError:
            print(f"열 번호가 유효한 범위를 벗어났습니다. 제목 생성을 건너뜁니다.")
            title = "Untitled"

        # Word 문서를 생성합니다
        doc = Document()

        # 제목을 추가합니다
        title_paragraph = doc.add_heading(title, level=1)
        set_paragraph_format(title_paragraph, font_size=12)  # 제목의 글자 크기 설정

        # 각 셀의 내용을 문단으로 추가합니다
        for col_name, cell in row.items():
            sanitized_content = sanitize_cell_content(cell)
            paragraph = doc.add_paragraph(f"{col_name} : {sanitized_content}")
            set_paragraph_format(paragraph, line_spacing=1, font_size=10)  # 내용의 줄 간격과 글자 크기 설정

        # 파일 이름을 생성하고 특수 문자를 제거합니다
        filename = sanitize_filename(title)
        word_file = os.path.join(output_dir, f"{filename}.docx")

        # Word 문서를 저장합니다
        doc.save(word_file)

# CSV 파일 경로와 저장할 디렉터리를 지정합니다
csv_file = r'c:\Users\kwonkt\Desktop\PythonStudy\니퐁내퐁 8기 신청서(응답) - 설문지 응답 시트1.csv'  # 실제 CSV 파일 경로
output_dir = r'C:\Users\kwonkt\Desktop\PythonStudy\csv로 워드파일 만들기 연습\설문모음8기_2_docs'  # 저장할 디렉터리 경로

# 함수를 호출하여 변환 작업을 수행합니다
csv_rows_to_words(csv_file, output_dir)